In [1]:
import os

os.chdir("../")

In [2]:
import glob

mscx_path_list = []
for item in glob.glob("./xmls/mscx/*"):
    mscx_path_list.append(os.path.abspath(item))
mscx_path_list.sort()

tstree_path_list = []
for item in glob.glob("./xmls/tstree/*"):
    tstree_path_list.append(os.path.abspath(item))
tstree_path_list.sort()

In [3]:
from alm.lyrics import *
from alm.melody import *
from alm.comparator import *
from alm.utils import io
import pprint

In [4]:
def calc_word_matched_rate(mscx_path_list: list, tstree_path_list: list,):
    parser = grammar_parser.GrammarParser("ja_ginza")

    res = []

    if len(mscx_path_list) != len(tstree_path_list):
        return res

    for i in range(len(mscx_path_list)):
        mscx_path = mscx_path_list[i]
        ts_path = tstree_path_list[i]

        lyrics_notes_dict = lyrics_extractor.extract_lyrics(mscx_path)
        doc = parser.parse(lyrics_notes_dict[lyrics_extractor.LYRICS_KEY])
        lyrics_tree = parser.to_tree(doc)
        words_notes_dict = {}
        associating_lyrics_melody.explore_words_in_tree(lyrics_tree, words_notes_dict)
        words_list = associating_lyrics_melody.associate_word_list_notes(words_notes_dict, lyrics_notes_dict)

        melody_tree = time_span_tree.tstree_xml_2_struct(ts_path)

        rate = word_matched_rate_calculator.calc_word_matched_rate(words_list, melody_tree)
        rate.section_name = io.get_file_name(mscx_path)

        res.append(rate)
    
    return res

In [5]:
word_matched_rates = calc_word_matched_rate(mscx_path_list, tstree_path_list)

In [6]:
word_matched_rate_dict = {}
for item in word_matched_rates:
    song_name = item.section_name[:-2]
    section = item.section_name[-1]

    if song_name not in word_matched_rate_dict:
        word_matched_rate_dict[song_name] = [-1, -1, -1, -1]

    if section == "A":
        word_matched_rate_dict[song_name][0] = item.total_words_number
        word_matched_rate_dict[song_name][1] = item.matched_words_number
    elif section == "S":
        word_matched_rate_dict[song_name][2] = item.total_words_number
        word_matched_rate_dict[song_name][3] = item.matched_words_number

res = [] 
for key, value in word_matched_rate_dict.items():
    value.insert(0, key)
    res.append(value)

io.output_csv(
    "./csv/word_matched_rate.csv",
    ["楽曲名", "Aメロの総単語数", "Aメロの一致した単語数", "サビの総単語数", "サビの一致した単語数"],
    res
)